In [ ]:
gcloud auth list

In [ ]:
gcloud config list project

In [ ]:
export ZONE=us-east4-a

##Task 1. Create a custom security role

In [ ]:
export CUSTOM_SECURIY_ROLE=orca_storage_editor_433

In [ ]:
gcloud config set compute/zone $ZONE

In [ ]:
nano role-definition.yaml
### OR vi role-definition.yaml

In [ ]:
title: "Custom-Security-Role"
description: "Permissions"
stage: "ALPHA"
includedPermissions:
- storage.buckets.get
- storage.objects.get
- storage.objects.list
- storage.objects.update
- storage.objects.create

# Ctrl + X -> Y -> Enter/Return
### OR :wq!

In [ ]:
gcloud iam roles create $CUSTOM_SECURIY_ROLE --project $DEVSHELL_PROJECT_ID --file role-definition.yaml

##Task 2. Create a service account

In [ ]:
export SERVICE_ACCOUNT=orca-private-cluster-152-sa

In [ ]:
gcloud iam service-accounts create $SERVICE_ACCOUNT --display-name "Orca Private Cluster Service Account"

##Task 3. Bind a custom security role to a service account

In [ ]:
gcloud projects add-iam-policy-binding $DEVSHELL_PROJECT_ID --member=serviceAccount:$SERVICE_ACCOUNT@$DEVSHELL_PROJECT_ID.iam.gserviceaccount.com --role roles/monitoring.viewer
gcloud projects add-iam-policy-binding $DEVSHELL_PROJECT_ID --member=serviceAccount:$SERVICE_ACCOUNT@$DEVSHELL_PROJECT_ID.iam.gserviceaccount.com --role roles/monitoring.metricWriter
gcloud projects add-iam-policy-binding $DEVSHELL_PROJECT_ID --member=serviceAccount:$SERVICE_ACCOUNT@$DEVSHELL_PROJECT_ID.iam.gserviceaccount.com --role roles/logging.logWriter
gcloud projects add-iam-policy-binding $DEVSHELL_PROJECT_ID --member serviceAccount:$SERVICE_ACCOUNT@$DEVSHELL_PROJECT_ID.iam.gserviceaccount.com --role projects/$DEVSHELL_PROJECT_ID/roles/$CUSTOM_SECURIY_ROLE


##Task 4. Create and configure a new Kubernetes Engine private cluster

In [ ]:
export CLUSTER_NAME=orca-cluster-958
export VPC_NAME=orca-build-vpc
export SUBNET_NAME=orca-build-subnet
export SERVICE_ACCOUNT=orca-private-cluster-152-sa

In [ ]:
gcloud compute instances describe orca-jumphost --zone $ZONE \
    --format="value(networkInterfaces[0].networkIP)"

In [ ]:
gcloud container clusters create $CLUSTER_NAME \
    --num-nodes=1 \
    --master-ipv4-cidr=172.16.0.64/28 \
    --network=$VPC_NAME \
    --subnetwork=$SUBNET_NAME \
    --enable-master-authorized-networks \
    --master-authorized-networks=192.168.10.2/32 \
    --enable-ip-alias \
    --enable-private-nodes \
    --enable-private-endpoint \
    --service-account=$SERVICE_ACCOUNT@$DEVSHELL_PROJECT_ID.iam.gserviceaccount.com \
    --zone=$ZONE

##Task 5. Deploy an application to a private Kubernetes Engine cluster

In [ ]:
export JUMPHOST_NAME=orca-jumphost

In [ ]:
gcloud compute ssh --zone $ZONE $JUMPHOST_NAME

In [ ]:
gcloud config set compute/zone us-east4-a

In [ ]:
sudo apt-get install google-cloud-sdk-gke-gcloud-auth-plugin

In [ ]:
gcloud container clusters get-credentials orca-cluster-958 --internal-ip

In [ ]:
kubectl create deployment hello-server --image=gcr.io/google-samples/hello-app:1.0

In [ ]:
kubectl expose deployment hello-server --name orca-hello-service --type LoadBalancer --port 80 --target-port 8080